In this notebook, we explore a Named Entity Recognition task using transformers. The task will involve finetuning the [ClinicalBert](https://huggingface.co/emilyalsentzer/Bio_ClinicalBERT) model.

In [8]:
! pip install pandas
! pip install datasets
! pip install transformers
! pip install torch

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.8 MB/s eta 0:00:0000:0100:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [9]:
import os
import itertools
import pandas as pd
import numpy as np
import datasets
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

## Load data

In [22]:
import pandas as pd

train = pd.read_csv("data/NCBI-disease-IOB/train.tsv", sep = "\t", header = None)
train.columns = ["tokens", "ner_tags"]
validation = pd.read_csv("data/NCBI-disease-IOB/devel.tsv", sep = "\t", header = None)
validation.columns = ["tokens", "ner_tags"]
test = pd.read_csv("data/NCBI-disease-IOB/test.tsv", sep = "\t", header = None)
test.columns = ["tokens", "ner_tags"]

Convert data to a datasets dictionary

In [26]:
from datasets import DatasetDict

df_ehr = DatasetDict({
    "train": Dataset.from_pandas(train),
    "validation": Dataset.from_pandas(validation),
    "test": Dataset.from_pandas(test)
})

df_ehr

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 135615
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 23959
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 24488
    })
})

In [ ]:
df_ehr["train"][1]

In [30]:
def get_all_tokens_and_ner_tags(directory):
    return pd.concat([get_tokens_and_ner_tags(os.path.join(directory, filename)) for filename in os.listdir(directory)]).reset_index().drop('index', axis=1)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split('\t')[0] for x in y] for y in split_list]
        entities = [[x.split('\t')[1][:-1] for x in y] for y in split_list] 
    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
def get_un_token_dataset(train_directory, test_directory):
    train_df = get_all_tokens_and_ner_tags(train_directory)
    test_df = get_all_tokens_and_ner_tags(test_directory)
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)

    return datasets.DatasetDict(
    {
        "train": train_dataset,
        "test": test_dataset,
    }
)

In [31]:
un_df = get_un_token_dataset('/workspaces/cogstack_project/data/NCBI-disease-IOB')

TypeError: get_un_token_dataset() missing 1 required positional argument: 'test_directory'

In [32]:
 split_list = [list(y) for x, y in itertools.groupby(train, lambda z: z == '\n') if not x]

In [33]:
split_list

[['tokens', 'ner_tags']]